In [21]:
import pandas as pd
import numpy as np
import PIL
from keras.applications import VGG16,imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.models import Model
from keras.layers import Dense
import numpy as np
import glob
import os

import pickle
import time


In [3]:
df = pd.read_table('Data/DeepFashion/list_attr_img.txt',skiprows=1,sep='\s+',header=None)

In [4]:
bbox = pd.read_table('Data/DeepFashion/list_bbox.txt',sep='\s+')

In [5]:
joined = bbox.join(df,lsuffix='image_name',rsuffix='0')

In [6]:
joined.shape[0]

289222

In [7]:
inputShape = (224, 224)
preprocess = imagenet_utils.preprocess_input

def preprocess_DF(bbox):
    image = crop_resize_DF(bbox)
    image = img_to_array(image)
    image = np.expand_dims(image,axis=0)
    
    return preprocess(image)

def crop_resize_DF(bbox):
    img = PIL.Image.open('./Data/DeepFashion/'+bbox['image_name'])
    lx = bbox['x_1']
    ly = bbox['y_1']
    ux = bbox['x_2']
    uy = bbox['y_2']
    
    img = img.crop((lx,ly,ux,uy))
    img = img.resize(inputShape, PIL.Image.ANTIALIAS)
    return img

In [ ]:
N = 20000
subset = np.random.randint(joined.shape[0],size=N)
proc_imgs = []
start = time.time()

for i in subset:
    proc_imgs.append(np.squeeze(preprocess_DF(joined.iloc[i])))
proc_imgs = np.array(proc_imgs)
print(f'{time.time()-start:.2f} s')

In [8]:
def generator(df, batch_size):
    while True:
        subset = np.random.randint(joined.shape[0],size=batch_size)
        proc_imgs = []
        for i in range(batch_size):
            proc_imgs.append(np.squeeze(preprocess_DF(df.iloc[i])))
    
        attr = (df.iloc[subset,6:].values>0)
        yield np.array(proc_imgs), attr

In [9]:
joined.head()

,image_name,x_1,y_1,x_2,y_2,0,1,2,3,4,...,991,992,993,994,995,996,997,998,999,1000
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,72,79,232,273,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,67,59,155,161,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,65,65,156,200,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,51,62,167,182,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,46,88,166,262,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [10]:
attr = (joined.iloc[:,6:].values>0)

In [11]:
class_weights = 1/(np.mean(attr,axis=0)*1000+1e-8)

In [12]:
base_model = VGG16(weights='imagenet')
x = base_model.get_layer('predictions').input
predictions = Dense(1000, activation='sigmoid',name='predictions')(x)

new_model = Model(inputs=base_model.input, outputs=predictions)

new_model.compile(optimizer='adam', loss='binary_crossentropy')

In [22]:
from keras import callbacks
import time
tbCB = callbacks.TensorBoard(log_dir="logs/{}".format(time.time()), histogram_freq=0, write_graph=False, write_images=False)
checkpointCB = callbacks.ModelCheckpoint("checkpoints/{}".format(time.time()))

new_model.fit_generator(generator(joined,64),steps_per_epoch=4519,epochs=1,verbose=1,callbacks=[tbCB,checkpointCB])

Epoch 1/1
 753/4519 [===>..........................] - ETA: 288052s - loss: 0.0566

KeyboardInterrupt: 

In [23]:
new_model.save_weights('Data/Trained Models/DF-Retrain 7-19.h5')

In [25]:
new_model.layers

In [13]:
new_model.load_weights('Data/Trained Models/DF-Retrain 7-19.h5')

In [14]:
feat_model = Model(inputs=new_model.input,outputs=new_model.get_layer('fc1').output)

In [15]:
import pickle
tops = pickle.load(open('Data/feature_matrix/tops_10000_df.p','rb'))

In [17]:
proc_tops = []
for index, row in tops.iloc[:100].iterrows():
    proc_tops.append(np.squeeze(preprocess_DF(row)))
proc_tops = np.array(proc_tops)

In [24]:
start = time.time()
fc6_retrain_tops = feat_model.predict(proc_tops)
#pickle.dump(fc6_retrain_tops,open('Data/feature_matrix/fc6_retrain_tops.p','wb'))
print(f'{time.time()-start:.2f} s')

41.66 s


In [25]:
fc6_retrain_tops

array([[  0.00000000e+00,   0.00000000e+00,   6.43580884e+13, ...,
          3.50490958e+12,   3.36793635e+13,   3.31692953e+12],
       [  0.00000000e+00,   0.00000000e+00,   8.35693849e+13, ...,
          4.28226589e+12,   4.37728009e+13,   4.90342344e+12],
       [  1.40497814e+11,   0.00000000e+00,   6.98879553e+13, ...,
          3.80038178e+12,   3.70454477e+13,   4.37352556e+12],
       ..., 
       [  2.72980312e+11,   0.00000000e+00,   1.17305464e+14, ...,
          5.27653457e+12,   6.19816503e+13,   9.39527556e+12],
       [  0.00000000e+00,   0.00000000e+00,   9.60219379e+13, ...,
          4.77244424e+12,   5.00884581e+13,   6.07404530e+12],
       [  0.00000000e+00,   0.00000000e+00,   6.35322216e+13, ...,
          3.36237927e+12,   3.33674059e+13,   3.34163241e+12]], dtype=float32)